In [1]:
import os
import CONSTANTS as c
import IMAGEUTILS as im
import XMLUTILS as xu
import numpy as np

In [2]:
def process_annotation(annotation, img_filepath):
    if len(annotation.findall('object')) > 0:
        return np.mean(im.imread(img_filepath), axis=(0,1))

In [4]:
means = []
#iterate through all files
for root, dirs, files in os.walk(c.BASE_DIR):
    for file in sorted(files):
        filepath = root + '/' + file

        # only allow images in the standardized format that have been annotated
        if file[0] == 'D' and file[-4:] == c.XML_EXT and 'xml_preds' not in filepath:
            annotation = xu.open_xml_file(filepath)
            img_filepath = filepath.replace('xmls', 'imgs').replace(c.XML_EXT, c.IMG_EXT)
            means.append(process_annotation(annotation, img_filepath))

In [5]:
means = [mean for mean in means if mean is not None]

In [6]:
np.mean(means, axis=(0))

array([  99.47443859,  121.87747586,  101.44344873])